In [149]:
f = open('Profile-20180716T115056', 'r')

In [150]:
import json

profile = json.load(f)

In [152]:
start = None
end = None

profile_events = []
trace_events = []

for row in profile:
    if row['ph'] != 'M':
        if start is None or int(row['ts']) < start:
            start = int(row['ts'])
        if end is None or int(row['ts']) > end:
            end = int(row['ts'])
    if row['ph'] in ['B', 'E', 'X']:
        trace_events.append(row)
    elif row['ph'] == 'I' and row['name'] == 'CpuProfile':
        profile_events.append(row)

In [153]:
def parse_nodes(data):
    nodes = {}
    for node in data['cpuProfile']['nodes']:
        node_id = node['id']
        function_name = node['callFrame']['functionName']
        url = node['callFrame']['url']
        line_number = node['callFrame']['lineNumber']
        children = node.get('children')
        hit_count = node.get('hitCount')
        nodes[node_id] = {'function_name': function_name, 'url': url, 'line_number': line_number, 'hit_count': hit_count, 'children': children}
    return nodes

In [154]:
import copy

def generate_stacks(node_id, nodes, stacks, current_stack):
    node = nodes[node_id] # break in case id doesn't exist
    if node['function_name'] == '':
        node['function_name'] = '(anonymous)'
    current_stack.append(node['function_name'])
    stacks[node_id] = current_stack
    if node['children']:
        for child in node['children']:
            generate_stacks(child, nodes, stacks, copy.copy(current_stack))
    del nodes[node_id]

In [155]:
def get_events(pid, tid, samples, time_deltas, start_time, stacks, program_id, idle_id, gc_id):
    events = []
    current_time = start_time
    last_sample = None
    last_sample_time = None
    for index, delta in enumerate(time_deltas):
        current_time += delta
        sample = samples[index]
        if sample != last_sample or (index + 1) == len(time_deltas):
            if sample not in (program_id, idle_id, gc_id):
                stack = stacks[sample]
                events.append({
                    'pid': pid,
                    'tid': tid,
                    'name': stack[len(stack) - 1],
                    'cat': 'JSSample',
                    'ph': 'X',
                    'ts': last_sample_time,
                    'dur': current_time - last_sample_time,
                    'stack': stack
                })
            last_sample = sample
            last_sample_time = current_time
    return events

In [156]:
def get_meta_ids(nodes):
    program_node_id = None
    idle_node_id = None
    gc_node_id = None
    for key, node in nodes.items():
        if node['function_name'] == '(program)':
            program_node_id = key
        elif node['function_name'] == '(idle)':
            idle_node_id = key
        elif node['function_name'] == '(garbage collector)':
            gc_node_id = key
    return program_node_id, idle_node_id, gc_node_id

In [157]:
js_events = []

for profile in profile_events:
    pid = profile['pid']
    tid = profile['tid']
    data = profile['args']['data']
    root_id = data['cpuProfile']['nodes'][0]['id']
    nodes = parse_nodes(data)
    program_id, idle_id, gc_id = get_meta_ids(nodes)
    stacks = {}
    generate_stacks(root_id, nodes, stacks, [])
    js_events += get_events(pid, tid, data['cpuProfile']['samples'], data['cpuProfile']['timeDeltas'], data['cpuProfile']['startTime'], stacks, program_id, idle_id, gc_id)


In [158]:
root = {'name': 'root', 'value': 0, 'children': []}
open_partial_slices = {}

# TODO: handle CPU time differences, where "E" comes before "B"

def get_child_slice(parent_slice, name):
    for index, child in enumerate(parent_slice['children']):
        if child['name'] == name:
            return parent_slice['children'].pop(index)
    return None

def insert_slice(parent_slice, new_slice):
    child_slice = get_child_slice(parent_slice, new_slice['name'])
    if child_slice is None:
        parent_slice['children'].append(new_slice)
    else:
        for child in new_slice['children']:
            insert_slice(child_slice, child)
        child_slice['value'] += new_slice['value']
        parent_slice['children'].append(child_slice)

def check_thread(pid, tid):
    if pid not in open_partial_slices:
        open_partial_slices[pid] = {}
    if tid not in open_partial_slices[pid]:
        open_partial_slices[pid][tid] = []

def begin_slice(pid, tid, cat, name, ts, tts):
    check_thread(pid, tid)
    open_partial_slices[pid][tid].append({'pid': pid, 'tid': tid, 'cat': cat, 'name': name, 'ts': ts, 'tts': tts, 'children': []})

def end_slice(pid, tid, ts, tts):
    partial_slice_count = len(open_partial_slices[pid][tid])
    if partial_slice_count > 0:
        current_slice = open_partial_slices[pid][tid].pop()
        current_slice['dur'] = ts - current_slice['ts']
        current_slice['tdur'] = tts - current_slice['tts']
        if current_slice['dur'] > 0:
            current_slice['value'] = current_slice['tdur'] / current_slice['dur']
        partial_slice_count = len(open_partial_slices[pid][tid])
        if partial_slice_count > 0:
            open_partial_slices[pid][tid][partial_slice_count - 1]['children'].append(current_slice)
        else:
            insert_slice(root, current_slice)
    else:
        raise Exception("end_slice called without an open slice")

In [159]:
from sortedcollection import SortedCollection

def event_ts(event):
    return event['ts']

sorted_profile = SortedCollection(key=event_ts)

for event in trace_events: 
    sorted_profile.insert_right(event)

print(len(sorted_profile))

for event in js_events:
    sorted_profile.insert_right(event)
    
print(len(sorted_profile))

142757
174462


In [162]:
# TODO: handle "sf" and "stack" properties on Duration Events

for row in iter(sorted_profile):
    if row['ph'] == 'B' or row['ph'] == 'E':
        if row['ph'] == 'B':
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row['tts'])
        elif row['ph'] == 'E':
            end_slice(row['pid'], row['tid'], row['ts'], row['tts'])
    elif row['ph'] == 'X':
        if 'dur' in row and row['dur'] > 0:
            if 'tdur' in row and row['tdur'] > 0:
                begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row['tts'])
                end_slice(row['pid'], row['tid'], row['ts'] + row['dur'], row['tts'] + row['tdur'])
            else:
                begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row['ts'])
                end_slice(row['pid'], row['tid'], row['ts'] + row['dur'], row['ts'] + row['dur'])

In [163]:
import json

with open('trace_event_v2.json', 'w') as file:
     file.write(json.dumps(root))